In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier

In [2]:
ds_demo = pd.read_csv("twitter_income.csv")
ds_demo.head(5)

,user_id,age,gender:female_gt_0_5,anxious:agree,anxious:strongly_agree,anxious:disagree,anxious:strongly_disagree,anxious:neither,children:no,children:yes,...,Topic_191,Topic_192,Topic_193,Topic_194,Topic_195,Topic_196,Topic_197,Topic_198,Topic_199,Topic_200
0,1103,30.700773,-0.050586,0.113558,0.034757,0.329889,0.221230,0.300567,0.682671,0.317329,...,0.143927,0.001070,0.001070,0.006956,0.042804,0.036918,0.019797,0.000000,0.093098,0.010166
1,8273,29.817867,0.376868,0.079798,0.030549,0.360824,0.285467,0.243363,0.693130,0.306870,...,0.164856,0.000000,0.001863,0.004967,0.053710,0.009624,0.005278,0.000621,0.066749,0.007762
2,9269,31.260406,0.204542,0.079508,0.026907,0.343969,0.273264,0.276351,0.835034,0.164966,...,0.136182,0.000000,0.001230,0.005841,0.014448,0.071011,0.004919,0.010452,0.043345,0.008300
3,10571,33.120257,0.164126,0.093635,0.037560,0.410242,0.185529,0.273034,0.837154,0.162846,...,0.128398,0.000000,0.002313,0.018508,0.020821,0.024870,0.010989,0.002313,0.074610,0.015616
4,11758,26.284088,-0.066739,0.103949,0.038573,0.239573,0.237358,0.380548,0.741889,0.258111,...,0.085206,0.000468,0.001404,0.004682,0.010768,0.020599,0.004682,0.001404,0.042603,0.015918


In [3]:
ds_demo.columns.get_loc("mean_income")

84

### Decision Tree Feature Selection(Top 20)

In [4]:
from sklearn.model_selection import train_test_split
y = ds_demo.loc[:,"mean_income"]
X = ds_demo.iloc[:,np.r_[1:84, 85:284]]
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=999)

In [5]:
X_train.head()

,age,gender:female_gt_0_5,anxious:agree,anxious:strongly_agree,anxious:disagree,anxious:strongly_disagree,anxious:neither,children:no,children:yes,education:degree,...,Topic_190,Topic_191,Topic_192,Topic_193,Topic_194,Topic_195,Topic_196,Topic_197,Topic_198,Topic_199
890,26.189070,0.703617,0.078929,0.033372,0.387177,0.243109,0.257412,0.734602,0.265398,0.328848,...,0.012674,0.048532,0.001236,0.001546,0.002782,0.029057,0.005564,0.007419,0.002473,0.057187
2161,29.070674,0.183706,0.073224,0.025529,0.392168,0.249821,0.259258,0.806966,0.193034,0.404329,...,0.023985,0.086716,0.000000,0.001845,0.016605,0.018450,0.060886,0.012915,0.000000,0.044280
1889,34.020592,0.670030,0.087392,0.024789,0.358172,0.221531,0.308116,0.761290,0.238710,0.380006,...,0.031918,0.123024,0.000930,0.001549,0.008677,0.031918,0.005888,0.007747,0.004958,0.066006
4984,27.239769,0.215531,0.118464,0.028897,0.431512,0.202958,0.218169,0.810287,0.189713,0.270806,...,0.020696,0.171214,0.002822,0.000941,0.000941,0.032926,0.019755,0.004704,0.001881,0.073377
3198,26.597661,0.034729,0.091383,0.025606,0.303174,0.254486,0.325351,0.750939,0.249061,0.384895,...,0.024715,0.168251,0.000951,0.002852,0.006654,0.062738,0.001901,0.004753,0.003802,0.071293


In [7]:
from sklearn.tree import DecisionTreeRegressor

# learn model
dt = DecisionTreeRegressor()
dt.fit(X_train,y_train)

# in sample accuracy
print ('In sample accuracy:',dt.score(X_train,y_train))

# out of sample accuracy
print ('Out of sample accuracy:',dt.score(X_test,y_test))

In sample accuracy: 1.0
Out of sample accuracy: -0.559578093965


In [9]:
# This time we'll use max_depth to control the complexity of the tree, still using the same train/test split as above,
# and optimize the parameter value using GridSearchCV.
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth':range(1,20)}
dt=DecisionTreeRegressor()
gr=GridSearchCV(dt,param_grid=param_grid)
rs=gr.fit(X_train,y_train)
print (rs.best_params_)
# print (roc_auc_score(np.array(y_test),rs.predict_proba(X_test)[:,1]))

{'max_depth': 3}


In [25]:
dt = DecisionTreeRegressor(max_depth=3)
dt.fit(X_train, y_train)
Feature_importance=pd.DataFrame([list(X_train.columns),list(dt.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 5 most important features in order
df_select = Feature_importance.sort_values(by="importance",ascending=False).iloc[:7,:]

In [26]:
df_select.head(7)

,variables,importance
245,Topic_163,0.441072
67,listed,0.193351
206,Topic_124,0.141791
85,Topic_3,0.0765151
72,enlish_tweets,0.0680371
114,Topic_32,0.0474928
177,Topic_95,0.031742


In [27]:
top_7 = list(df_select.variables)

In [28]:
top_7

['Topic_163',
 'listed',
 'Topic_124',
 'Topic_3',
 'enlish_tweets',
 'Topic_32',
 'Topic_95']